<a href="https://colab.research.google.com/github/btomlinson237/Ukraine-War-Reserach/blob/master/Tomlinson_Stock_Analysis_with_Back_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

In [57]:
# The following creates a dataframe of daily market data of SPEU, going back 1 year; by default, the data will collect a year of daily opening prices,
    # daily closing prices, daily price highs, daily price lows, adjusted closing prices, and trading volume
fullData = yf.download(tickers = 'SPEU', period = '1y', interval = '1d', prepost = True)

[*********************100%***********************]  1 of 1 completed


In [58]:
fullData.tail() # Displays data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-27,37.590000,37.759998,37.590000,37.680000,37.680000,16300
2023-02-28,37.630001,37.650002,37.330002,37.349998,37.349998,24900
2023-03-01,37.700001,37.720001,37.400002,37.549999,37.549999,66100
2023-03-02,37.290001,37.639999,37.220001,37.590000,37.590000,24500
2023-03-03,37.790001,38.098801,37.680000,38.070000,38.070000,83936


In [59]:
def MACDcalc(data):
  # MACD is a technical indicator that tracks the momentum of trends by demonstrating the relationship between two moving averages of the price of a stock
  # This calculation, and most calculations, of MACD track the relationship subtract a short (usually 12-period) exponential moving average of the price
      # by a long (usually 26-period) exponential moving average of the price.
  # MACD is usually compared against a "signal line" that represents a 9-period exponential moving average of the price, and the intersections between MACD
      # and the signal line indicate a shift from a bear market to a bull market, or vice-versa

  ShortEMA = data.Close.ewm(span=12, adjust = False).mean() # Calculates the 12-period EMA
  LongEMA = data.Close.ewm(span=26, adjust=False).mean() # Calculates the 26-period EMA
  MACD = ShortEMA - LongEMA # Subtracts short EMA by long EMA to arrive at MACD

  nineWeekEMA = MACD.ewm(span=9, adjust = False).mean() # Calculates the 9-period EMA to be compared against MACD
  data['MACD'] = MACD # adds MACD calculations to SPEU dataframe
  data['9-Week EMA'] = nineWeekEMA # Adds 9-week EMA to SPEU dataframe

MACDcalc(fullData)
fullData.tail()

,Open,High,Low,Close,Adj Close,Volume,MACD,9-Week EMA
Date,,,,,,,,
2023-02-27,37.590000,37.759998,37.590000,37.680000,37.680000,16300,0.147182,0.291772
2023-02-28,37.630001,37.650002,37.330002,37.349998,37.349998,24900,0.104746,0.254367
2023-03-01,37.700001,37.720001,37.400002,37.549999,37.549999,66100,0.086260,0.220745
2023-03-02,37.290001,37.639999,37.220001,37.590000,37.590000,24500,0.073984,0.191393
2023-03-03,37.790001,38.098801,37.680000,38.070000,38.070000,83936,0.101813,0.173477


In [60]:
# ADX is a technical indicator that tracks the strength of a market trend. Note that it indicates only the magnitude of the trend, and not the direction;
    # thus, a higher ADX indicates only the presence of either a bearish or bullish market.

def adxCalc(data):
  high = data['High'] # Sets high equal to the daily price peaks of SPEU
  low = data['Low'] # Sets low equal to the daily price troughs of SPEU
  close = data['Close'] # Sets close equal to the daily closing prices of SPEU
  lookback = 14 # Sets the "lookback" period equal to 14 periods, which is standard

  plus_dm = high.diff() # Establishes the positive Directional Movement (+DM) of SPEU using price peaks
  minus_dm = low.diff() # Establishes the negative Directional Movement (-DM) of SPEU using price troughs
  plus_dm[plus_dm < 0] = 0 # Normalizes positive Directional Movement to only be positive
  minus_dm[minus_dm > 0] = 0 # Normalizes negative Directional Movement to only be negative

  # Creates 3 dataframes to measure three differences
  tr1 = pd.DataFrame(high - low) # Calculates difference between a day's price peak and the same day's price trough
  tr2 = pd.DataFrame(abs(high - close.shift(1))) # Calculates absolute difference between day's price high and next day's closing price
  tr3 = pd.DataFrame(abs(low - close.shift(1))) # Calculates absolute difference between day's price low and next day's closing price

  frames = [tr1, tr2, tr3] # Merges the 3 differences into one dataframe (3 columns)

  # Calculates the true range (TR) by selecting the maximum of the three differences for each index(day)
  tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1) # consolidates the dataframe into one column representing the TR (max of differences)

  # Average true range (ATR) is calculated by taking the average true range of the lookback period (14 periods)
  atr = tr.rolling(lookback).mean()


  plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr) # Establishes positive Directional Index(+DI) = EMA of +DM / ATR
  minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr)) # Establishes negative Directional Index(-DI) = EMA of -DM / ATR

  dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100 # Calculates Directional Index(DI) using +DI and -DI 
  adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback # Calculates the Average Directional Index(ADX) using DI and the lookback period
  adx_smooth = adx.ewm(alpha = 1/lookback).mean() # Smooths ADX to provide more accurate values by using a custom moving average

  data['Avg Directional Index'] = adx_smooth # Adds (smoothed) ADX calculations to SPEU dataframe

adxCalc(fullData)
fullData.tail()

,Open,High,Low,Close,Adj Close,Volume,MACD,9-Week EMA,Avg Directional Index
Date,,,,,,,,,
2023-02-27,37.590000,37.759998,37.590000,37.680000,37.680000,16300,0.147182,0.291772,26.074913
2023-02-28,37.630001,37.650002,37.330002,37.349998,37.349998,24900,0.104746,0.254367,25.157329
2023-03-01,37.700001,37.720001,37.400002,37.549999,37.549999,66100,0.086260,0.220745,23.768512
2023-03-02,37.290001,37.639999,37.220001,37.590000,37.590000,24500,0.073984,0.191393,22.578134
2023-03-03,37.790001,38.098801,37.680000,38.070000,38.070000,83936,0.101813,0.173477,21.153151


In [61]:
# Visualizes data from two indicators with respect to SPEU

# Creates 2 separate plots (since MACD values don't visually scale well with ADX values) - a MACD plot and an ADX plot with a shared x-axis(time)
SPEUfig = make_subplots(rows = 2, cols =1, shared_xaxes = True, subplot_titles = ("SPEU Live Moving Average Convergence/Divergence", "SPEU Live Average Directional Movement Index"))

# Establishes MACD and EMASignal series for upper plot
SPEUfig.append_trace(go.Scatter(x=fullData.index, y = fullData['MACD'], line=dict(color='blue', width = .8), name = 'MACD'), row =1, col = 1)
SPEUfig.append_trace(go.Scatter(x=fullData.index, y = fullData['9-Week EMA'], line = dict(color='red', width = .8), name = '9-Week EMA'), row = 1, col = 1)

# Establishes ADX series for lower plot
SPEUfig.append_trace(go.Scatter(x=fullData.index, y = fullData['Avg Directional Index'], line = dict(color='green', width = .8), name = "Average Directional Index"), row = 2, col = 1)

# Allows viewer to dynamically adjust the time interval for the indicator of interest; applies to both plots simultaneously
SPEUfig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=14, label="1 Week", step="day", stepmode="backward"),
            dict(count=40, label="1 Month", step="day", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=1, label="1 Day", step="day", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Sets axis titles for each plot
SPEUfig['layout']['xaxis2']['title']= 'Date'
SPEUfig['layout']['yaxis']['title'] = 'MACD Value'
SPEUfig['layout']['yaxis2']['title'] = 'ADX Value'

# Sets title for the visual containing both plots
SPEUfig.update_layout(title_text='SPDR Portfolio Europe ETF: Live MACD and ADX Performance', xaxis_rangeslider_visible = False, height = 600)

# Displays the visual containing both plots
SPEUfig.show()

In [62]:
testData = yf.download('SPEU', start = "2022-03-01", end = "2022-05-31")
MACDcalc(testData)
adxCalc(testData)
testData.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,MACD,9-Week EMA,Avg Directional Index
Date,,,,,,,,,
2022-05-23,36.130001,36.439999,36.110001,36.400002,35.424919,11800,-0.502701,-0.678911,21.307522
2022-05-24,36.290001,36.490002,36.139999,36.380001,35.405453,9300,-0.406728,-0.624474,19.753744
2022-05-25,36.099998,36.540001,36.099998,36.369999,35.395718,3600,-0.327699,-0.565119,18.345893
2022-05-26,36.459999,36.900002,36.459999,36.840000,35.853127,10400,-0.224554,-0.497006,17.091859
2022-05-27,37.139999,37.299999,37.090000,37.290001,36.291077,18000,-0.105286,-0.418662,16.424303


In [63]:
def setBuySell(data):
  data['Buy/Sell Signal'] = 0
  data.loc[data['MACD'] < data['9-Week EMA'], 'Buy/Sell Signal'] = 1
  data.loc[data['MACD'] > data['9-Week EMA'], 'Buy/Sell Signal'] = 0

  test = data[data["Buy/Sell Signal"] != 0]
  data['Buy/Sell Signal'] = 0
  data['Buy/Sell Signal'] = np.where(data['MACD'] < data['9-Week EMA'], 1, 0 )
  data['Buy/Sell Signal'] = np.where(data['MACD'] > data['9-Week EMA'], -1, data['Buy/Sell Signal'])

setBuySell(testData)
testData.tail()

,Open,High,Low,Close,Adj Close,Volume,MACD,9-Week EMA,Avg Directional Index,Buy/Sell Signal
Date,,,,,,,,,,
2022-05-23,36.130001,36.439999,36.110001,36.400002,35.424919,11800,-0.502701,-0.678911,21.307522,-1
2022-05-24,36.290001,36.490002,36.139999,36.380001,35.405453,9300,-0.406728,-0.624474,19.753744,-1
2022-05-25,36.099998,36.540001,36.099998,36.369999,35.395718,3600,-0.327699,-0.565119,18.345893,-1
2022-05-26,36.459999,36.900002,36.459999,36.840000,35.853127,10400,-0.224554,-0.497006,17.091859,-1
2022-05-27,37.139999,37.299999,37.090000,37.290001,36.291077,18000,-0.105286,-0.418662,16.424303,-1


In [64]:
testMarketData = go.Candlestick(x=testData.index, open = testData['Open'], high = testData['High'], low=testData['Low'], close = testData['Close'], name = 'Market Data')
testMACD = go.Scatter(x = testData.index, y = testData['MACD'], line=dict(color='blue', width = .8), name = 'MACD')
test9wEMA = go.Scatter(x = testData.index, y = testData['9-Week EMA'], line=dict(color='red', width = .8), name = '9-Week EMA')


testFig = go.Figure()

testFig = make_subplots(rows = 2, cols =1, shared_xaxes = True, subplot_titles = ("Price Data", "SPEU Live Average Directional Movement Index"))

testFig.add_trace(testMarketData, row = 1, col =1)
testFig.add_trace(testMACD, row =2, col =1)
testFig.add_trace(test9wEMA, row = 2, col =1)

testFig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(buttons=list([
                     dict(count=15, label="15m", step="minute", stepmode="backward"),
                     dict(count=45, label="45m", step="minute", stepmode="backward"),
                     dict(count=1, label="HTD", step="hour", stepmode="todate"),
                     dict(count=3, label="3h", step="hour", stepmode="backward"),
                     dict(step="all")
                 ])))
testFig.update_layout(title = 'SPEU Historical Stock Data')

testFig.add_trace(go.Scatter(x=testData.index[testData["Buy/Sell Signal"]==1], y = testData["Close"][testData["Buy/Sell Signal"]==1], 
                             mode = "markers", marker_color = "darkgreen", marker_symbol = "arrow-up", marker_size = 10,
                             name = "Buy Signal"), row = 1, col =1)

testFig.add_trace(go.Scatter(x=testData.index[testData["Buy/Sell Signal"]==-1], y = testData["Close"][testData["Buy/Sell Signal"]==-1], 
                             mode = "markers", marker_color = "darkred", marker_symbol = "arrow-down", marker_size = 10,
                             name = "Sell Signal"), row = 1, col = 1)
testFig.show()



In [65]:
def MACD_returns(data):
  entryPrices = []
  entryDates = []
  exitPrices = []
  exitDates = []
  inMarket = False

  for i in range(data.shape[0]):
    if (inMarket == False) and (data.iloc[i]["Buy/Sell Signal"] == 1):
      inMarket = True
      entryPrices.append((data.iloc[i]["Close"]))
      entryDates.append(data.iloc[i].name)
    if (inMarket == True) and (data.iloc[i]["Buy/Sell Signal"] == -1):
      inMarket = False
      exitPrices.append(data.iloc[i]["Close"])
      exitDates.append(data.iloc[i].name)
  
  if (len(entryPrices) > len(exitPrices)):
    exitPrices.append(data.iloc[-1]["Close"])
    exitDates.append(data.iloc[-1].name)

  return entryPrices, exitPrices, entryDates, exitDates

testEntryPrices, testExitPrices, testEntryDates, testExitDates = MACD_returns(testData)

sales = pd.DataFrame({'Entry Prices': testEntryPrices, 'Exit Prices': testExitPrices})

sales

,Entry Prices,Exit Prices
0,37.48,38.529999
1,38.48,36.150002


In [66]:
def profitsFrame(entryPrices, exitPrices):
  sales = pd.DataFrame({'Entry Prices': entryPrices, 'Exit Prices': exitPrices})
  
  profits = sales['Profits'] = sales['Exit Prices'] - sales['Entry Prices']
  
  relativeProfits = sales['Relative Profits'] = sales['Profits'] / sales['Entry Prices']

  averageProfit = relativeProfits.mean()

  return averageProfit, sales



testAverageProfit, testSales = profitsFrame(testEntryPrices, testExitPrices)

print("Average Return: ")
print("%.0f%%" % (100 * testAverageProfit))
testSales
    


Average Return: 
-2%


,Entry Prices,Exit Prices,Profits,Relative Profits
0,37.48,38.529999,1.049999,0.028015
1,38.48,36.150002,-2.329998,-0.060551
